In [1]:
import gensim

C:\Anaconda2\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [34]:
import numpy as np

with open("glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [53]:
import string
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords as sw
from nltk.corpus import wordnet as wn
from nltk import wordpunct_tokenize
from nltk import WordNetLemmatizer
from nltk import sent_tokenize
from nltk import pos_tag

from sklearn.base import BaseEstimator, TransformerMixin


class NLTKPreprocessor(BaseEstimator, TransformerMixin):

    def __init__(self, stopwords=None, punct=None,
                 lower=True, strip=True):
        self.lower      = lower
        self.strip      = strip
        self.stopwords  = stopwords or set(sw.words('english'))
        self.punct      = punct or set(string.punctuation)
        self.lemmatizer = WordNetLemmatizer()
        stopwords_custom = pd.read_csv("stopwords1.csv")
        self.stopwords_custom = stopwords_custom.ix[:,0].map(lambda x:x.lower())

    def fit(self, X, y=None):
        return self

    def inverse_transform(self, X):
        return [" ".join(doc) for doc in X]

    def transform(self, X):
        return [
            list(self.tokenize(doc)) for doc in X
        ]
    
    ## Removing HTML tags
    def cleanhtml(self, raw_html):
        cleanr = re.compile(r'<[^>]+>')
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext
    
    ## Removing URL
    def strip_URL(self,text):
        return re.sub(r'(https|http)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    
    ## Removing Hashtags and @tags 
    def strip_all_tags(self,text):
        entity_prefixes = ['@','#']
        for separator in  string.punctuation:
            if separator not in entity_prefixes :
                text = text.replace(separator,' ')
        words = []
        for word in text.split():
            word = word.strip()
            if word:
                if word[0] not in entity_prefixes:
                    words.append(word)
        return ' '.join(words)
    
    ## Removing special characters 
    def strip_special_chars(self,text):
        return re.sub('\W+',' ', text)
    
    ## Removing extra white spaces
    def strip_extra_white_spaces(self,text):
        return re.sub(' +',' ',text).strip()
    
    ## Removing extra new lines
    def strip_extra_new_lines(self,text):
        return re.sub("\n+","\n", text)

    def tokenize(self, document):
        document = self.cleanhtml(document)
        document = self.strip_URL(document)
        document = self.strip_all_tags(document)
        document = self.strip_special_chars(document)
        document = self.strip_extra_white_spaces(document)
        document = self.strip_extra_new_lines(document)
        # Break the document into sentences
        for sent in sent_tokenize(document):
            # Break the sentence into part of speech tagged tokens
            for token, tag in pos_tag(wordpunct_tokenize(sent)):
                # Apply preprocessing to the token
                token = token.lower() if self.lower else token
                
                # If stopword, ignore token and continue
                if token.lower in self.stopwords_custom.tolist():
                    continue

                # If punctuation, ignore token and continue
                if all(char in self.punct for char in token):
                    continue

                # Lemmatize the token and yield
                lemma = self.lemmatize(token, tag)
                yield lemma

    def lemmatize(self, token, tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

In [54]:

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.itervalues().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [67]:
#do not consider

class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.itervalues().next())
#global defaultdict 
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [90]:
! pip install -U scikit-learn

  Found existing installation: scikit-learn 0.17.1
    Uninstalling scikit-learn-0.17.1:
      Successfully uninstalled scikit-learn-0.17.1
  Rolling back uninstall of scikit-learn


    DEPRECATION: Uninstalling a distutils installed project (scikit-learn) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only partially uninstall the project.
Exception:
Traceback (most recent call last):
  File "c:\anaconda2\lib\site-packages\pip\basecommand.py", line 215, in main
    status = self.run(options, args)
  File "c:\anaconda2\lib\site-packages\pip\commands\install.py", line 342, in run
    prefix=options.prefix_path,
  File "c:\anaconda2\lib\site-packages\pip\req\req_set.py", line 784, in install
    **kwargs
  File "c:\anaconda2\lib\site-packages\pip\req\req_install.py", line 851, in install
    self.move_wheel_files(self.source_dir, root=root, prefix=prefix)
  File "c:\anaconda2\lib\site-packages\pip\req\req_install.py", line 1064, in move_wheel_files
    isolated=self.isolated,
  File "c:\anaconda2\lib\site-packages\pip\wheel.py", line 345, in move_wheel_files
    clobber(source, lib_dir, 

In [102]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\anaconda2\lib\site-packages


In [106]:
import sklearn
print (sklearn.__version__)

0.17.1


In [107]:
from sklearn.cross_validation import KFold

In [94]:
from sklearn.model_selection import KFold

ImportError: No module named model_selection

In [122]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split as tts
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import KFold

def identity(arg):
    """
    Simple identity function works as a passthrough.
    """
    return arg

def build_and_evaluate(X, y,
    classifier, outpath=None, verbose=True):

    def build(classifier, X, y=None):
        print(classifier)
        """
        Inner build function that builds a single model.
        """
        if isinstance(classifier, type):
            classifier = classifier()

        model = Pipeline([
            ('preprocessor', NLTKPreprocessor()),
            ("Word-2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("LinearSVC", classifier)
        ])

        model.fit(X, y)
        return model

    # Label encode the targets
    labels = LabelEncoder()
    y = labels.fit_transform(y)

    # Begin evaluation
    if verbose: print("Building for evaluation")
    #X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)
    #model = build(classifier, X_train, y_train)
    
    #X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)
    #model = build(classifier, X_train, y_train)
    
    #kf = KFold(n_splits=3)
    #X_train, X_test, y_train, y_test = kf.split(X, y)
    #model = build(classifier, X_train, y_train)
    
    #kf = KFold(len(X), n_folds=3)
    #for train, test in kf.split(X):
        #X_train = np.array(X)[train]
        #X_test = np.array(X)[test]
    
    #for train, test in kf.split(y):
        #y_train = np.array(y)[train]
        #y_test = np.array(y)[test]
    
    
    kf = KFold(2066, n_folds=3) 

    for train_index, test_index in kf:
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    model = build(classifier, X_train, y_train)

    
    
    if verbose:
#         print("Evaluation model fit in {:0.3f} seconds".format(secs))
        print("Classification Report:\n")

    y_pred = model.predict(X_test)
    print(clsr(y_test.tolist(), y_pred.tolist()))

    if verbose:
        print("Building complete model and saving ...")
    model = build(classifier, X, y)
    model.labels_ = labels

#     if verbose:
#          print("Complete model fit in {:0.3f} seconds".format(secs))

    if outpath:
        with open(outpath, 'wb') as f:
            pickle.dump(model, f)

        print("Model written out to {}".format(outpath))

    return model

In [123]:
X = posts["posts_p"]
y = posts["TAG"]
print("SGD Classifier")
model = build_and_evaluate(X,y,SGDClassifier)
print("SVM classifier")
model = build_and_evaluate(X,y,LinearSVC)
#print("MultinomialNB classifier")
#model = build_and_evaluate(X,y,MultinomialNB)

SGD Classifier
Building for evaluation
('TRAIN:', array([ 689,  690,  691, ..., 2063, 2064, 2065]), 'TEST:', array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

In [40]:
X = [['Berlin', 'London'],
     ['cow', 'cat'],
     ['pink', 'yellow']]
y = ['capitals', 'animals', 'colors']
etree_w2v.fit(X, y)

# never before seen words!!!
test_X = [['dog'], ['red'], ['Madrid']]

print etree_w2v.predict(test_X)

['animals' 'colors' 'capitals']


In [41]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import csr_matrix

# example data
X_train = np.array([[1., 1.], [2., 3.], [4., 0.]])
print "X_train: \n", X_train
# transform to scipy.sparse.csr.csr_matrix to be consistent with output from load_svmlight_file
X_train_crs = csr_matrix(X_train)
print "X_train_crs: \n", X_train_crs   
# no problem to run TfidfTransformer() on this csr matrix to get a transformed csr matrix
#transformer = TfidfTransformer()
#tfidf = transformer.fit_transform(X_train)
#print "tfidf: \n", tfidf
# How do I use the HashingVectorizer with X_train_crs ?
hv = HashingVectorizer(n_features=2)

X_train: 
[[1. 1.]
 [2. 3.]
 [4. 0.]]
X_train_crs: 
  (0, 0)	1.0
  (0, 1)	1.0
  (1, 0)	2.0
  (1, 1)	3.0
  (2, 0)	4.0


In [11]:
import pandas as pd